In [9]:
# 1. Import libraries
import pandas as pd
import numpy as np
from IPython.display import display

pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 120)

In [10]:
# 2. Load dataset
raw_path = "../data/raw_dataset.csv"  # make sure you have the correct file here
df = pd.read_csv(raw_path)
print("Loaded dataset:", raw_path)

Loaded dataset: ../data/raw_dataset.csv


In [11]:
# 3. Initial exploration (before)
print("=== BEFORE CLEANING ===")
print("Shape:", df.shape)
display(df.head(8))
print("\nInfo:")
display(df.info())
print("\nDescribe (numeric):")
display(df.describe(include='all'))
print("\nMissing values per column:")
display(df.isna().sum())
print("\nDuplicate rows count:", df.duplicated().sum())

=== BEFORE CLEANING ===
Shape: (891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S



Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


None


Describe (numeric):


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Braund, Mr. Owen Harris",male,NaN,NaN,NaN,347082,NaN,B96 B98,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN



Missing values per column:


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


Duplicate rows count: 0


In [12]:
# 4. Handle Missing Values

# Copy dataset to preserve original in memory
df_clean = df.copy()

# Option A: Simple approach
df_clean['Age_simple_fill'] = df_clean['Age'].fillna(df_clean['Age'].median())

# Option B: Improved — group-based median using Pclass & Sex
group_med = df_clean.groupby(['Pclass','Sex'])['Age'].median().reset_index()
group_med = group_med.rename(columns={'Age':'Age_group_median'})

# Map group medians back to missing Age values
def fill_age_groupwise(row):
    if pd.isna(row['Age']):
        median = group_med[
            (group_med['Pclass']==row['Pclass']) &
            (group_med['Sex']==row['Sex'])
        ]['Age_group_median']
        if len(median)>0 and not pd.isna(median.values[0]):
            return median.values[0]
        else:
            return df_clean['Age'].median()
    else:
        return row['Age']

df_clean['Age'] = df_clean.apply(fill_age_groupwise, axis=1)

# Embarked: fill with mode
df_clean['Embarked'] = df_clean['Embarked'].fillna(df_clean['Embarked'].mode()[0])

# Cabin: derive HasCabin and deck letter; Cabin is sparse so we'll keep HasCabin and drop Cabin
df_clean['HasCabin'] = df_clean['Cabin'].notna().astype(int)
# Extract deck (first letter) if cabin not null
df_clean['CabinDeck'] = df_clean['Cabin'].dropna().apply(lambda x: x.split()[0][0] if len(x)>0 else np.nan)
df_clean['CabinDeck'] = df_clean['CabinDeck'].fillna('Unknown')

# If you prefer to drop Cabin entirely:
df_clean = df_clean.drop(columns=['Cabin'])

In [13]:
# 5. Remove duplicates
before_dup = df_clean.shape[0]
df_clean = df_clean.drop_duplicates()
after_dup = df_clean.shape[0]
print(f"Dropped {before_dup - after_dup} duplicate rows")

# 6. Standardize formats
df_clean['Name'] = df_clean['Name'].astype(str).str.strip()
df_clean['Sex'] = df_clean['Sex'].astype(str).str.lower().str.strip()
df_clean['Embarked'] = df_clean['Embarked'].astype(str).str.upper().str.strip()
df_clean['Ticket'] = df_clean['Ticket'].astype(str).str.strip()

# Convert Pclass to categorical
df_clean['Pclass'] = df_clean['Pclass'].astype('category')

Dropped 0 duplicate rows


In [ ]:
# 7. Outlier Detection & Treatment (IQR method)
def iqr_bounds(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return lower, upper

# Fare
fare_lower, fare_upper = iqr_bounds(df_clean['Fare'].dropna())
print("Fare bounds:", fare_lower, fare_upper)
fare_outliers = df_clean[(df_clean['Fare'] < fare_lower) | (df_clean['Fare'] > fare_upper)]
print("Fare outliers count:", fare_outliers.shape[0])

# Age
age_lower, age_upper = iqr_bounds(df_clean['Age'].dropna())
print("Age bounds:", age_lower, age_upper)
age_outliers = df_clean[(df_clean['Age'] < age_lower) | (df_clean['Age'] > age_upper)]
print("Age outliers count:", age_outliers.shape[0])


# Cap outliers at bounds (winsorize-like)
df_capped = df_clean.copy()
df_capped['Fare'] = np.where(df_capped['Fare'] > fare_upper, fare_upper, df_capped['Fare'])
df_capped['Fare'] = np.where(df_capped['Fare'] < fare_lower, fare_lower, df_capped['Fare'])
df_capped['Age'] = np.where(df_capped['Age'] > age_upper, age_upper, df_capped['Age'])
df_capped['Age'] = np.where(df_capped['Age'] < age_lower, age_lower, df_capped['Age'])

Fare bounds: -26.724 65.6344
Fare outliers count: 116
Age bounds: -0.25 57.75
Age outliers count: 33


In [15]:
# 8. Before/after snapshots and summary stats
print("=== AFTER CLEANING (capped) ===")
print("Shape:", df_capped.shape)
display(df_capped.head(8))
print("\nMissing values per column after cleaning:")
display(df_capped.isna().sum())
print("\nDescribe (numeric) after cleaning:")
display(df_capped.describe())

=== AFTER CLEANING (capped) ===
Shape: (891, 14)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Age_simple_fill,HasCabin,CabinDeck
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,22.0,0,Unknown
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,65.6344,C,38.0,1,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,26.0,0,Unknown
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,35.0,1,C
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,35.0,0,Unknown
5,6,0,3,"Moran, Mr. James",male,25.0,0,0,330877,8.4583,Q,28.0,0,Unknown
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,S,54.0,1,E
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,S,2.0,0,Unknown



Missing values per column after cleaning:


PassengerId        0
Survived           0
Pclass             0
Name               0
Sex                0
Age                0
SibSp              0
Parch              0
Ticket             0
Fare               0
Embarked           0
Age_simple_fill    0
HasCabin           0
CabinDeck          0
dtype: int64


Describe (numeric) after cleaning:


,PassengerId,Survived,Age,SibSp,Parch,Fare,Age_simple_fill,HasCabin
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,28.893850,0.523008,0.381594,24.046813,29.361582,0.228956
std,257.353842,0.486592,12.732352,1.102743,0.806057,20.481625,13.019697,0.420397
min,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000,0.420000,0.000000
25%,223.500000,0.000000,21.500000,0.000000,0.000000,7.910400,22.000000,0.000000
50%,446.000000,0.000000,26.000000,0.000000,0.000000,14.454200,28.000000,0.000000
75%,668.500000,1.000000,36.000000,1.000000,0.000000,31.000000,35.000000,0.000000
max,891.000000,1.000000,57.750000,8.000000,6.000000,65.634400,80.000000,1.000000


In [16]:
# 9. Save cleaned dataset
out_path = "../data/cleaned_dataset.csv"
df_capped.to_csv(out_path, index=False)
print("Saved cleaned dataset to", out_path)

Saved cleaned dataset to ../data/cleaned_dataset.csv
